In [1]:
import torch
import numpy as np

In [2]:
x_seeds = np.array([(0,0), (1,0), (0,1), (1,1)], dtype=np.float32)
y_seeds = np.array([0, 1, 1, 0])

In [3]:
N = 1000
idxs = np.random.randint(0, 4, N)

In [4]:
X = x_seeds[idxs]
Y = y_seeds[idxs]

In [5]:
X = X + np.random.normal(scale = 0.25, size=X.shape)

In [6]:
class shallow_neural_network():
    def __init__(self, num_input_features, num_hiddens):
        self.num_input_features = num_input_features
        self.num_hiddens = num_hiddens
        
        # numpy 구현과 달리 troch tensor 사용
        self.W1 = torch.randn((num_hiddens, num_input_features), requires_grad=True)
        self.b1 = torch.randn(num_hiddens, requires_grad=True)
        self.W2 = torch.randn(num_hiddens, requires_grad=True)
        self.b2 = torch.randn(1, requires_grad=True)
        
        self.tanh = torch.nn.Tanh()
        self.sigmoid = torch.nn.Sigmoid()
        
    def predict(self, x):
        z1 = torch.matmul(self.W1, x) + self.b1
        a1 = self.tanh(z1)
        z2 = torch.matmul(self.W2, a1) + self.b2
        a2 = self.sigmoid(z2)
        return a2

In [7]:
model = shallow_neural_network(2, 3)

In [8]:
def train(X, Y, model, lr=0.1):
    
    m = len(X)
    cost = 0.0
    for x, y in zip(X, Y):
        
        x_torch = torch.FloatTensor(x)
        a2 = model.predict(x_torch)
        
        if y==1:
            loss = -torch.log(a2+0.0001)
        else:
            loss = -torch.log(1.0001-a2)
            
        loss.backward() # gradient 계산 
        cost+=loss.item()
        
    with torch.no_grad(): # model parameter update
        model.W1 -= lr*model.W1.grad/m
        model.b1 -= lr*model.b1.grad/m
        model.W2 -= lr*model.W2.grad/m
        model.b2 -= lr*model.b2.grad/m
        
    model.W1.requires_grad = True # 다시 model parameter tracking
    model.b1.requires_grad = True
    model.W2.requires_grad = True
    model.b2.requires_grad = True
        
    return cost/m

In [9]:
for epoch in range(100):
    cost = train(X, Y, model, 1.0)
    if epoch%10==0:
        print(epoch, cost)

0 1.225547370173037
10 0.6786052489280701
20 0.8443314267247916
30 0.7470117318034172
40 0.785558275103569
50 0.7663633985221386
60 0.718724917806685
70 0.6981206574104726
80 0.6649162597879767
90 0.6634832704737782


In [10]:
model.predict(torch.Tensor((1,1)))[0].item()

0.6158723831176758

In [11]:
model.predict(torch.Tensor((1,0)))[0].item()

0.6158723831176758

In [12]:
model.predict(torch.Tensor((0,1)))[0].item()

0.9791457653045654

In [13]:
model.predict(torch.Tensor((0,0)))[0].item()

0.6158723831176758